System found in **Pymablock: an algorithm and a package for
quasi-degenerate perturbation theory**: arXiv:2404.03728v1

In [1]:
# Objects for defining the Hamiltonian
from SymPT import RDSymbol, RDBasis, BosonOp, Dagger, Block, Operator
# Objects for obtaining the Effective Hamiltonian
from SymPT import EffectiveFrame

# Extra utils
from SymPT import display_dict, group_by_operators

# Import sympy
import sympy as sp

# Setup

In [2]:
# ---------------- Defining the symbols ------------------
# Order 0
omega_t = RDSymbol('omega_t', order=0, positive=True, real=True)
omega_r = RDSymbol('omega_r', order=0, positive=True, real=True)
alpha   = RDSymbol('alpha', order=0, positive=True, real=True)
# Order 1
g = RDSymbol('g', order=1, positive=True, real=True)

# ----------------- Defining the basis -------------------
# Boson basis transmon: Infinite bosonic Hilbert space
a_t  = BosonOp('a_t')
ad_t = Dagger(a_t)
# Boson basis resonator: Infinite bosonic Hilbert space
a_r  = BosonOp('a_r')
ad_r = Dagger(a_r)

# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0 = -omega_t * ad_t * a_t + omega_r * ad_r * a_r + sp.Rational(1,2) * alpha * ad_t * ad_t * a_t * a_t
display(H0)
# Interaction Hamiltonian V
V = -g * (ad_t - a_t) * (ad_r - a_r)
display(V)

alpha*Dagger(a_t)**2*a_t**2/2 + omega_r*Dagger(a_r)*a_r - omega_t*Dagger(a_t)*a_t

-g*(Dagger(a_t) - a_t)*(Dagger(a_r) - a_r)

In [3]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V)

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│  a_t   │ Bosonic │      ∞      │
├────────┼─────────┼─────────────┤
│  a_r   │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




# Standard Schrieffer-Wolff Transformation

In [4]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=False, mask=None)
# Obtaining the result in the dict_operator form
H_eff_SWT = Eff_frame.get_H(return_form='operator')
# display_dict(H_eff_SWT)

Solving for each order: 100%|████████████| 2/2 [00:00<00:00,  2.61it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|████| 20/20 [00:00<00:00, 432.23it/s]


In [8]:
# substitution dictionary for transmon ground state
subs_nt0 = {
    a_t : 0,
    ad_t : 0
}

# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [9]:
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [25]:
grouped_H2 = group_by_operators(H2.subs(subs_nt0).expand())
display_dict({k : v.simplify() for k,v in grouped_H2.items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Multiblock

In [54]:
# Deffining the mask
mask = Block(inf=a_r*a_t) +  Block(inf=ad_r*a_t) + Block(inf=a_r**2) + Block(inf=a_t**2) + Block(inf=a_t**2*a_r**2)+ Block(inf=ad_t**2*a_r**2)
# Calculate the effective model using the Mask routine up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=False, mask=mask)
H_eff_Mask = Eff_frame.get_H(return_form='operator')
# display_dict(H_eff_Mask)

The perturbative interaction will be added to the full Hamiltonian


Solving for each order: 100%|████████████| 2/2 [00:01<00:00,  1.30it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|██████| 6/6 [00:00<00:00, 522.17it/s]


In [55]:
# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [56]:
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [58]:
nt = RDSymbol("N_t")
grouped_H2 = group_by_operators(H2.expand())
display_dict({k : v.subs({ad_t*a_t : nt}).simplify().factor().collect((nt,alpha,omega_t,omega_r,2)) for k,v in grouped_H2.items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [72]:
keys = list(grouped_H2.keys())
idx = 1
display(keys[idx])
grouped_H2[keys[idx]].subs({ad_t*a_t : nt}).simplify().factor().collect((nt,alpha,omega_t,omega_r,2,nt*alpha))

Dagger(a_t)*a_t*Dagger(a_r)*a_r

2*alpha*g**2*(4*N_t**6*alpha**6 + 2*N_t**4*alpha**4*omega_r**2 + N_t**4*alpha**4*(13*alpha**2 + 50*omega_t**2) + 6*N_t**2*alpha**4*omega_r**2 + 66*N_t**2*alpha**4*omega_t**2 - 14*N_t*alpha*omega_t**5 + alpha**4*(N_t**2*alpha**2 - 2*N_t*alpha*omega_t + omega_r**2 + omega_t**2) + alpha**3*(-6*N_t**3*alpha**3 + 17*N_t**2*alpha**2*omega_t - 4*N_t*alpha*omega_r**2 - 16*N_t*alpha*omega_t**2 + 3*omega_r**2*omega_t + 5*omega_t**3) + 2*alpha**2*omega_r**2*omega_t**2 + alpha*(-12*N_t**5*alpha**5 + 55*N_t**4*alpha**4*omega_t - 4*N_t**3*alpha**3*omega_r**2 - 100*N_t**3*alpha**3*omega_t**2 + 6*N_t**2*alpha**2*omega_r**2*omega_t + 90*N_t**2*alpha**2*omega_t**3 + 8*N_t*alpha*omega_r**4 - 40*N_t*alpha*omega_t**4 - 5*omega_r**4*omega_t - 2*omega_r**2*omega_t**3 + 7*omega_t**5) + 2*omega_r**6 + omega_r**4*(-8*N_t**2*alpha**2 - 3*alpha**2 - 2*omega_t**2) + 2*omega_t**6 + omega_t**4*(40*N_t**2*alpha**2 + 9*alpha**2 - 2*omega_r**2) + omega_t**3*(-60*N_t**3*alpha**3 - 40*N_t*alpha**3 + 4*N_t*alpha*omega_r**

# Full-diagonalization

In [36]:
# Calculate the effective model using the Full Diagonalization routine up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=True, mask=None)
H_eff_FD = Eff_frame.get_H(return_form='dict_operator')

The perturbative interaction will be added to the full Hamiltonian


Solving for each order: 100%|████████████| 2/2 [00:01<00:00,  1.27it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Converting to dictionary (operator) form: 100%|█| 9/9 [00:00<00:00, 533


In [37]:
# Corrections to the Hamiltonian for each order
H0, H2 = [H_correction for H_correction in Eff_frame.corrections.values()]

In [38]:
display_dict(group_by_operators(H0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [39]:
nt = RDSymbol("N_t")
grouped_H2 = group_by_operators(H2.expand())
display_dict({k : v.subs({ad_t*a_t : nt}).simplify().factor() for k,v in grouped_H2.items()})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>